<center>
    <p style="text-align:center">
        <img alt="arize llama-index logos" src="https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/arize_llamaindex.png" width="400">
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating and Improving a LlamaIndex Search and Retrieval Application</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service using LlamaIndex. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Download an pre-indexed knowledge base of the Arize documentation and run a LlamaIndex application
- Visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using cosine similarity, Euclidean distance, and LLM-assisted ranking metrics

Parts of this notebook require an [OpenAI API key](https://platform.openai.com/account/api-keys) to run. If you don't have an OpenAI key, you can still run Phoenix by skipping cells preceded by the 💭 emoji.


## Chatbot Architecture

Your chatbot was built using LlamaIndex's low-level API. The architecture of your chatbot is shown below and can be explained in five steps.

![llama-index chatbot architecture](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/llama_index_chatbot_architecture.png)

1. The user sends a query about Arize to your service.
1. `langchain.embeddings.OpenAIEmbeddings` makes a request to the OpenAI embeddings API to embed the user query using the `text-embedding-ada-002` model.
1. `llama_index.retrievers.RetrieverQueryEngine` does a similarity search against the entries of your index knowledge base for the two most similar pieces of context by cosine similarity.
1. `llama_index.indices.query.ResponseSynthesizer` generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI chat completions API with the `gpt-3.5-turbo`.
2. The response is returned to the user.

Phoenix makes your search and retrieval system *observable* by capturing the inputs and outputs of these steps for analysis, including:

- your query embeddings
- the retrieved context and similarity scores for each query
- the generated response that is return to the user

With that overview in mind, let's dive into the notebook.

## 1. Install Dependencies and Import Libraries

Install Phoenix and LlamaIndex.

In [ ]:
!pip install -q arize-phoenix gcsfs llama-index

Import libraries.

In [1]:
from functools import reduce
import hashlib
import json
import logging
import os
import sys
import tempfile
import textwrap
from tqdm import tqdm
from typing import Any, Dict, List, Tuple
import urllib
import zipfile

from gcsfs import GCSFileSystem
from llama_index.graph_stores.simple import SimpleGraphStore
from langchain.chat_models import ChatOpenAI
from llama_index import StorageContext, load_index_from_storage
from llama_index.callbacks import CallbackManager, OpenInferenceCallbackHandler
from llama_index.callbacks.open_inference_callback import as_dataframe
from llama_index.embeddings.base import BaseEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.indices.query.schema import QueryBundle
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.response.schema import Response
from llama_index import ServiceContext, LLMPredictor
from llama_index import StorageContext, load_index_from_storage
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.response.schema import Response
import numpy as np
import numpy.typing as npt
import openai
import pandas as pd
import phoenix as px
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)


pd.set_option("display.max_colwidth", None)


## 2. Configure Your OpenAI API Key

💭 Configure your OpenAI API key.

In [ ]:
openai_api_key = "copy paste your api key here"
assert openai_api_key != "copy paste your api key here", "❌ Please set your OpenAI API key"
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key


## 3. Download Your Knowledge Base

Download your pre-built index from cloud storage and instantiate your storage context.

In [2]:
file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
    graph_store=SimpleGraphStore(),  # pass default graph store to prevent unauthorized request to GCS
)


Download and unzip a pre-built knowledge base index consisting of chunks of the Arize documentation.

## 4. Run Your Question-Answering Service

💭 Start a LlamaIndex application from your downloaded index.

In [3]:
callback_handler = OpenInferenceCallbackHandler()
service_context = ServiceContext.from_defaults(
    llm_predictor=LLMPredictor(llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)),
    embed_model=OpenAIEmbedding(model="text-embedding-ada-002"),
    callback_manager=CallbackManager(handlers=[callback_handler]),
)
index = load_index_from_storage(
    storage_context,
    service_context=service_context,
)
query_engine = index.as_query_engine()


💭 Ask a few questions of your question-answering service and view the responses.

In [7]:
max_line_length = 80
for query in [
    "What's the difference between primary and baseline datasets?",
    "How do I send in extra metadata with each record?",
    "How does Arize's surrogate explainability model work?",
]:
for query in query_df[":feature.text:prompt"].to_list():
    print("Query")
    print("=====")
    print()
    print(textwrap.fill(query, max_line_length))
    print()
    response = query_engine.query(query)
    print("Response")
    print("========")
    print()
    print(textwrap.fill(str(response), max_line_length))
    print()

Query
=====

How do I use the SDK to upload a ranking model?

Response

To use the SDK to upload a ranking model, you can follow these steps:  1.
Install the SDK: Start by installing the SDK for the programming language you
are using. The SDK provides the necessary tools and libraries to interact with
the Arize platform.  2. Import the SDK: Import the SDK into your project by
adding the appropriate import statements or dependencies.  3. Authenticate: Use
your Arize API key to authenticate your SDK client. This key can be obtained
from your Arize account.  4. Create a model: Use the SDK to create a model
object that represents your ranking model. Provide the necessary information
such as the model name, description, and any other relevant details.  5. Upload
the model: Use the SDK's upload function to upload your ranking model to the
Arize platform. This function typically takes the model object created in the
previous step as a parameter.  6. Verify the upload: After the upload is
comp

## 5. Load Your Data Into Pandas Dataframes

To use Phoenix, you must load your data into pandas dataframes. First, load your knowledge base into a dataframe.

In [11]:
def storage_context_to_dataframe(storage_context: StorageContext) -> pd.DataFrame:
    """Converts the storage context to a pandas dataframe.

    Args:
        storage_context (StorageContext): Storage context containing the index
        data.

    Returns:
        pd.DataFrame: The dataframe containing the index data.
    """
    document_ids = []
    document_texts = []
    document_embeddings = []
    docstore = storage_context.docstore
    vector_store = storage_context.vector_store
    for node_id, node in docstore.docs.items():
        document_ids.append(node.hash)  # use node hash as the document ID
        document_texts.append(node.text)
        document_embeddings.append(np.array(vector_store.get(node_id)))
    return pd.DataFrame(
        {
            "document_id": document_ids,
            "text": document_texts,
            "text_vector": document_embeddings,
        }
    )


database_df = storage_context_to_dataframe(storage_context)
database_df = database_df.drop_duplicates(subset=["text"])
database_df.head()


,document_id,text,text_vector
0,86dacab3db386524f430c7dcdb08d55a580bc705630861bb9ce7b255d1df9d59,\nAccess tutorials of what's possible with Arize below:\n\n1. Model Type Examples\n2. Explainability Tutorials\n3. Cloud Storage Examples\n\n,"[-0.000973847636487335, 0.000874826917424798, 0.012771095149219036, -0.0024746579583734274, 0.006788518745452166, 0.011021440848708153, 0.016849027946591377, -0.013322167098522186, -0.01576066017150879, -0.04863210767507553, 0.0020492991898208857, 0.0037197363562881947, -0.023158803582191467, 0.00039328463026322424, -0.003361539449542761, -0.0017617085250094533, 0.003912611398845911, 0.013914569281041622, 0.03207239508628845, -0.02147803269326687, -0.016173964366316795, -0.01239223312586546, -0.02178112231194973, -0.015168257988989353, 0.0011589734349399805, 0.023489445447921753, 0.02442626841366291, -0.020072799175977707, 0.00850717443972826, -0.014768730849027634, 0.02654789574444294, 0.014038560912013054, 0.013659698888659477, -0.042790744453668594, -0.005744926165789366, 0.0018064831383526325, 0.02817355841398239, 0.022318419069051743, 0.003936721011996269, -0.010649467818439007, 0.033257197588682175, -0.016036197543144226, 0.0007155326311476529, 0.005011311732232571, -0.007060610689222813, 0.01162073202431202, -0.004267364274710417, -0.005741482134908438, -0.00934755988419056, -0.028104674071073532, 0.010470368899405003, 0.01386635098606348, -0.025280430912971497, 0.004907985683530569, -0.004325915593653917, 0.01137274969369173, -0.00628911005333066, 0.028765961527824402, -0.005538274068385363, 0.011042105965316296, 0.0034820865839719772, 0.0002385108673479408, -0.025652404874563217, -0.0013062129728496075, -0.014134998433291912, 0.013721694238483906, -0.014713623560965061, 0.01664237678050995, -0.005004423204809427, 0.0038540600799024105, 0.026079485192894936, 0.00795610249042511, -0.0048735435120761395, -0.033202093094587326, 0.010298158973455429, -0.028848621994256973, -0.0051456354558467865, -0.012268241494894028, -0.012337125837802887, 0.011338307522237301, -0.01657349243760109, 0.004267364274710417, -0.014589632861316204, -0.011393414810299873, 0.01478250790387392, 0.01162073202431202, 0.025762619450688362, 0.012523112818598747, -0.004257031716406345, -0.006199560593813658, 0.015526454895734787, -0.0011245313799008727, -0.0023558330722153187, 0.027016308158636093, -0.0014612020459026098, 0.01671125926077366, -0.00034958633477799594, 0.03226526826620102, -0.004629005212336779, -0.01576066017150879, ...]"
1,a909d43e39685381fca64b936735b972fd1433137148bda0a5f7e43f17d51c61,"\n{% hint style=""info"" %}\nYour model type determines which performance metrics are available to you. Learn more about model types here.\n{% endhint %}\n\nModel TypePandas BatchPython Single RecordCSVParquet Binary Classification (Only Classification Metrics)Colab LinkColab LinkDownload File*Open Parquet Reader HereBinary Classification (Classification, AUC/Log Loss Metrics) Colab LinkColab LinkDownload File*Open Parquet Reader HereBinary Classification (Classification, AUC/Log Loss, Regression) Colab LinkColab LinkDownload FileDownload File*Open Parquet Reader HereMulticlass Classification (Only Classification Metrics)Colab LinkColab LinkDownload File*Open Parquet Reader HereMulticlass Classification (Classification, AUC/Log Loss Metrics)Colab LinkColab LinkDownload File*Open Parquet Reader HereRegressionColab LinkColab LinkDownload FileDownload File*Open Parquet Reader HereTimeseries Forecasting Colab LinkColab LinkDownload File*Open Parquet Reader HereRanking with Relevance ScoreColab LinkColab LinkDownload File*Open Parquet Reader HereRanking with Single LabelColab LinkColab LinkDownload File*Open Parquet Reader HereRanking with Multiple LabelsColab LinkColab LinkDownload File*Open Parquet Reader HereNLP Classification Colab LinkNLP Named Entity Recognition (NER)Colab LinkCV Classification Colab LinkTabular Classification w/ Embeddings Colab LinkObject DetectionColab LinkLarge Language Models (LLMs)Colab Link

The columns of your dataframe are:
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text, computed during the LlamaIndex build using "text-embedding-ada-002" from OpenAI

💭 Your query data is saved in a buffer on the callback handler you defined in step 4.

In [8]:
query_data_buffer = callback_handler.flush_query_data_buffer()
sample_query_df = as_dataframe(query_data_buffer)
sample_query_df

,:id.id:,:timestamp.iso_8601:,:feature.text:prompt,:feature.[float].embedding:prompt,:prediction.text:response,:feature.[str].retrieved_document_ids:prompt,:feature.[float].retrieved_document_scores:prompt
0,4b7b62d0-fd55-4c47-83ca-2b35405b0774,2023-07-25T13:10:15.139446,What's the difference between primary and baseline datasets?,"[0.011037665419280529, 0.015439888462424278, 0.006235888227820396, -0.013263748958706856, -0.03145056590437889, 0.017038103193044662, 0.0020958720706403255, 0.0059076836332678795, -0.030480222776532173, -0.03370518982410431, -0.004259525798261166, 0.029937971383333206, -0.01611056737601757, 0.0021636534947901964, -0.005147818475961685, 0.01947823353111744, 0.021418921649456024, -0.013463526032865047, 0.0020387929398566484, -0.01766597293317318, -0.02936718240380287, 0.00017056830984074622, -0.007819832302629948, -0.011765423230826855, 0.004744697827845812, 0.010630976408720016, 9.712917744764127e-06, -0.009047032333910465, 0.007020725402981043, -0.014576567336916924, 0.011130418628454208, 0.00801247451454401, -0.013962967321276665, -0.0013770326040685177, 0.0016231860499829054, 0.000490077305585146, 0.0027219580952078104, 0.0037672186736017466, 0.03844275325536728, 0.0109021021053195, 0.0012691174633800983, 0.03182158246636391, -0.004330874420702457, -0.002663095248863101, 0.021176334470510483, 0.015297191217541695, 0.01262874435633421, -0.0023652140516787767, -0.02498636208474636, 0.015825172886252403, 0.0009676686022430658, 0.010638111270964146, -0.023830512538552284, -0.00605751620605588, -0.0198349766433239, -0.010224288329482079, -0.00853332132101059, 0.007527302484959364, 0.003763651242479682, -0.004577028099447489, -0.0010720163118094206, 0.010531088337302208, -0.01722360961139202, 0.010110130533576012, -0.02655603736639023, -0.019749358296394348, 0.005422511603683233, 0.017694512382149696, 0.002345592947676778, 0.011051935143768787, 0.013513470068573952, 0.01655293069779873, -0.012336214073002338, 0.004473572131246328, 0.024272873997688293, -0.01939261332154274, 0.0009542906773276627, -0.006043246481567621, 0.010017376393079758, 0.012157841585576534, 0.01211503241211176, -0.006482041906565428, -0.025271758437156677, -0.012521721422672272, 0.015982139855623245, 0.027626270428299904, 0.009938892908394337, 0.01758035272359848, 0.009204000234603882, -0.00178818020503968, 0.007719944231212139, 0.009104111231863499, 0.04834597185254097, 0.009674902074038982, -0.01973508857190609, 0.0046055675484240055, 0.012443237006664276, 0.017437655478715897, -0.008961413986980915, -0.018136873841285706, ...]","The primary dataset refers to the current data that you are working with or evaluating. It is the dataset that you want to compare and analyze in terms of model performance or any other metrics.\n\nOn the other hand, a baseline dataset is a reference dataset that is used for comparison with the primary dataset. It can be from previous periods in production, training data, validation data, or any other relevant dataset. The purpose of a baseline dataset is to provide a benchmark or reference point to measure the performance of the primary dataset against. It helps in identifying any changes or improvements in the model's performance and understanding the root causes of any performance degradation.\n\nIn summary, the primary dataset is the current data being analyzed, while the baseline dataset is a reference dataset used for comparison and benchmarking purposes.","[c55dc0b1919e3dc9e51077722110a540e5f201f65105d3c5f51a3aab1cce875e, 2023685f6dc07bf2bc98d7c6aa810ffd7a798147a38ea3ec7e006df60f2050c1]","[0.851456493535011, 0.8494208316976325]"
1,fd6050d4-15aa-4eef-a706-e1473111cc2a,2023-07-25T13:10:19.638752,How do I send in extra metadata with each record?,"[-0.0028620215598493814, -0.006446495186537504, 0.0014466408174484968, -0.02693912573158741, -0.00015553993580397218, 0.02830067090690136, -0.020048044621944427, -0.02635560743510723, -0.01932559348642826, -0.014351787976920605, 0.007266200613230467, 0.02756

The columns of the dataframe are:
- **text:** the query text
- **text_vector:** the embedding representation of the query, captured from LlamaIndex at query time
- **response:** the final response from the LlamaIndex application
- **context_text_0:** the first retrieved context from the knowledge base
- **context_similarity_0:** the cosine similarity between the query and the first retrieved context
- **context_text_1:** the second retrieved context from the knowledge base
- **context_similarity_1:** the cosine similarity between the query and the first retrieved context

Download a dataframe containing query data.

In [40]:
query_df = pd.read_parquet(
    # "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/llama-index/arize-docs/query_data_complete.parquet",
    "query_data_complete.parquet"
)
query_df.head()

,index,:id.id:,:timestamp.iso_8601:,:feature.text:prompt,:feature.[float].embedding:prompt,:prediction.text:response,:feature.[str].retrieved_document_ids:prompt,:feature.[float].retrieved_document_scores:prompt,retrieved_document_text_0,openai_relevance_0,retrieved_document_text_1,openai_relevance_1,user_feedback
0,0,4b7b62d0-fd55-4c47-83ca-2b35405b0774,2023-07-25T13:10:15.139446,What's the difference between primary and baseline datasets?,"[0.011037665419280529, 0.015439888462424278, 0.006235888227820396, -0.013263748958706856, -0.03145056590437889, 0.017038103193044662, 0.0020958720706403255, 0.0059076836332678795, -0.030480222776532173, -0.03370518982410431, -0.004259525798261166, 0.029937971383333206, -0.01611056737601757, 0.0021636534947901964, -0.005147818475961685, 0.01947823353111744, 0.021418921649456024, -0.013463526032865047, 0.0020387929398566484, -0.01766597293317318, -0.02936718240380287, 0.00017056830984074622, -0.007819832302629948, -0.011765423230826855, 0.004744697827845812, 0.010630976408720016, 9.712917744764127e-06, -0.009047032333910465, 0.007020725402981043, -0.014576567336916924, 0.011130418628454208, 0.00801247451454401, -0.013962967321276665, -0.0013770326040685177, 0.0016231860499829054, 0.000490077305585146, 0.0027219580952078104, 0.0037672186736017466, 0.03844275325536728, 0.0109021021053195, 0.0012691174633800983, 0.03182158246636391, -0.004330874420702457, -0.002663095248863101, 0.021176334470510483, 0.015297191217541695, 0.01262874435633421, -0.0023652140516787767, -0.02498636208474636, 0.015825172886252403, 0.0009676686022430658, 0.010638111270964146, -0.023830512538552284, -0.00605751620605588, -0.0198349766433239, -0.010224288329482079, -0.00853332132101059, 0.007527302484959364, 0.003763651242479682, -0.004577028099447489, -0.0010720163118094206, 0.010531088337302208, -0.01722360961139202, 0.010110130533576012, -0.02655603736639023, -0.019749358296394348, 0.005422511603683233, 0.017694512382149696, 0.002345592947676778, 0.011051935143768787, 0.013513470068573952, 0.01655293069779873, -0.012336214073002338, 0.004473572131246328, 0.024272873997688293, -0.01939261332154274, 0.0009542906773276627, -0.006043246481567621, 0.010017376393079758, 0.012157841585576534, 0.01211503241211176, -0.006482041906565428, -0.025271758437156677, -0.012521721422672272, 0.015982139855623245, 0.027626270428299904, 0.009938892908394337, 0.01758035272359848, 0.009204000234603882, -0.00178818020503968, 0.007719944231212139, 0.009104111231863499, 0.04834597185254097, 0.009674902074038982, -0.01973508857190609, 0.0046055675484240055, 0.012443237006664276, 0.017437655478715897, -0.008961413986980915, -0.018136873841285706, ...]","The primary dataset refers to the current data that you are working with or evaluating. It is the dataset that you want to compare and analyze in terms of model performance or any other metrics.\n\nOn the other hand, a baseline dataset is a reference dataset that is used for comparison with the primary dataset. It can be from previous periods in production, training data, validation data, or any other relevant dataset. The purpose of a baseline dataset is to provide a benchmark or reference point to measure the performance of the primary dataset against. It helps in identifying any changes or improvements in the model's performance and understanding the root causes of any performance degradation.\n\nIn summary, the primary dataset is the current data being analyzed, while the baseline dataset is a reference dataset used for comparison and benchmarking purposes.","[c55dc0b1919e3dc9e51077722110a540e5f201f65105d3c5f51a3aab1cce875e, 2023685f6dc07bf2bc98d7c6aa810ffd7a798147a38ea3ec7e006df60f2050c1]","[0.851456493535011, 0.8494208316976325]","\nA model baseline refers to a reference dataset used for comparison with your current data. A model baseline can be from training, validation, or previous periods in production. Baselines help compare model changes, enable detailed analysis, and help identify t

In addition to the columns of the previous dataframe, this data has a few additional fields:

- **user_feedback:** approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)
- **openai_relevance_0:** a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the first retrieved document is relevant to the query
- **openai_relevance_1:** a binary classification (relevant vs. irrelevant) by GPT-4 predicting whether the second retrieved document is relevant to the query

We'll go over how to compute the relevance classifications in section 6.

The query and database datasets are drawn from different distributions; the queries are short questions while the database entries are several sentences to a paragraph. The embeddings from OpenAI's "text-embedding-ada-002" capture these differences and naturally separate the query and context embeddings into distinct regions of the embedding space. When using Phoenix, you want to "overlay" the query and context embedding distributions so that queries appear close to their retrieved context in the Phoenix point cloud. To achieve this, we compute a centroid for each dataset that represents an average point in the embedding distribution and center the two distributions so they overlap.

In [ ]:
database_embedding_column_name = "text_vector"
database_centroid = database_df[database_embedding_column_name].mean()
database_df[database_embedding_column_name] = database_df[database_embedding_column_name].apply(
    lambda x: x - database_centroid
)
query_embedding_column_name = ":feature.[float].embedding:prompt"
query_centroid = query_df[query_embedding_column_name].mean()
query_df[query_embedding_column_name] = query_df[query_embedding_column_name].apply(
    lambda x: x - query_centroid
)

## 6. Run LLM-Assisted Evaluations

Cosine similarity and Euclidean distance are reasonable proxies for retrieval quality, but they don't always work perfectly. A novel idea is to use LLMs to measure retrieval quality by simply asking the LLM whether each retrieved document is relevant to the corresponding query.

💭 Use OpenAI to predict whether each retrieved document is relevant or irrelevant to the query.

⚠️ It's strongly recommended to use GPT-4 for evaluations if you have access.

In [38]:
EVALUATION_SYSTEM_MESSAGE = "You will be given a query and a reference text. You must determine whether the reference text contains an answer to the input query. Your response must be binary (0 or 1) and should not contain any text or characters aside from 0 or 1. 0 means that the reference text does not contain an answer to the query. 1 means the reference text contains an answer to the query."
QUERY_CONTEXT_PROMPT_TEMPLATE = """# Query: {query}

# Reference: {reference}

# Binary: """
num_retrieved_documents = 2


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def evaluate_query_and_retrieved_context(query: str, context: str, model_name: str) -> str:
    prompt = QUERY_CONTEXT_PROMPT_TEMPLATE.format(
        query=query,
        reference=context,
    )
    response = openai.ChatCompletion.create(
        messages=[
            {"role": "system", "content": EVALUATION_SYSTEM_MESSAGE},
            {"role": "user", "content": prompt},
        ],
        model=model_name,
    )
    return response["choices"][0]["message"]["content"]


def evaluate_retrievals(
    retrievals_data: Dict[str, str],
    model_name: str,
) -> List[str]:
    responses = []
    for query, retrieved_context in tqdm(retrievals_data.items()):
        response = evaluate_query_and_retrieved_context(query, retrieved_context, model_name)
        responses.append(response)
    return responses


def process_binary_responses(
    binary_responses: List[str], binary_to_string_map: Dict[int, str]
) -> List[str]:
    """
    Parse binary responses and convert to the desired format
    converts them to the desired format. The binary_to_string_map parameter
    should be a dictionary mapping binary values (0 or 1) to the desired
    string values (e.g. "irrelevant" or "relevant").
    """
    processed_responses = []
    for binary_response in binary_responses:
        try:
            binary_value = int(binary_response.strip())
            processed_response = binary_to_string_map[binary_value]
        except (ValueError, KeyError):
            processed_response = None
        processed_responses.append(processed_response)
    return processed_responses


# sample_query_df = query_df.head(10).copy()
# evaluation_model_name = "gpt-3.5-turbo"  # change to GPT-4 if you have access
evaluation_model_name = "gpt-4"
document_id_to_text = dict(zip(database_df["document_id"], database_df["text"]))
query_texts = sample_query_df[":feature.text:prompt"].to_list()
for retrieved_document_index in range(num_retrieved_documents):
    retrieved_document_ids = [
        doc_ids[retrieved_document_index]
        for doc_ids in sample_query_df[":feature.[str].retrieved_document_ids:prompt"].to_list()
    ]
    retrieved_document_texts = [document_id_to_text[doc_id] for doc_id in retrieved_document_ids]
    retrievals_data = dict(zip(query_texts, retrieved_document_texts))
    raw_responses = evaluate_retrievals(retrievals_data, evaluation_model_name)
    processed_responses = process_binary_responses(raw_responses, {0: "irrelevant", 1: "relevant"})
    sample_query_df[
        f"retrieved_document_text_{retrieved_document_index}"
    ] = retrieved_document_texts
    sample_query_df[f"openai_relevance_{retrieved_document_index}"] = processed_responses


sample_query_df[
    [
        ":feature.text:prompt",
        "retrieved_document_text_0",
        "retrieved_document_text_1",
        "openai_relevance_0",
        "openai_relevance_1",
    ]
].rename(columns={":feature.text:prompt": "prompt"}).head()


100%|██████████| 155/155 [01:43<00:00,  1.49it/s]
/var/folders/5r/4zhmkbjs6rg6kbz87n1kctpm0000gp/T/ipykernel_5136/2354930402.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_query_df[
/var/folders/5r/4zhmkbjs6rg6kbz87n1kctpm0000gp/T/ipykernel_5136/2354930402.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_query_df[f"openai_relevance_{retrieved_document_index}"] = processed_responses
100%|██████████| 155/155 [01:41<00:00,  1.53it/s]
/var/folders/5r/4zhmkbjs6rg6kbz87n1kctpm0000gp/T/ipyk

,prompt,retrieved_document_text_0,retrieved_document_text_1,openai_relevance_0,openai_relevance_1
0,What's the difference between primary and baseline datasets?,"\nA model baseline refers to a reference dataset used for comparison with your current data. A model baseline can be from training, validation, or previous periods in production. Baselines help compare model changes, enable detailed analysis, and help identify the root causes of performance degradation. **A model baseline is required in Arize**.\n\nBy default, model baselines use a moving time range from your model's **production data**. This data spans a period of two weeks and is delayed by three days. However, you have the flexibility to choose a baseline from either production data or pre-production data, which includes both training and validation sets.\n\nProduction Baseline#production-baselinePre-Production Baseline#pre-production-baselines\n\n","\nA baseline is the reference data or benchmark used to compare model performance against for monitoring purposes. Baselines can be training data, validation data, prior time periods of production data, a previous model version, among others.\n\n",irrelevant,irrelevant
1,How do I send in extra metadata with each record?,"\nIn your container metadata, add an entry with the key as `arize_ingestion_key` and the provided tag value.&#x20;\n\n* **In Arize UI:** Copy the `arize_ingestion_key` value.\n* **In Azure UI:** Navigate to your Container -> Settings -> Metadata.\n\nClick on Metadata and fill out the key value pair defined in the Arize UI\n\n",\nNavigate to the custom metric editor in the top navigation bar of any model. Click on 'Custom Metrics' to uncover the editor.&#x20;\n\n,relevant,irrelevant
2,How does Arize's surrogate explainability model work?,"\nArize is the machine learning observability platform for ML practitioners to monitor, troubleshoot, and explain models. Data Science and ML Engineering teams of all sizes (from individuals to enterprises) use Arize to:\n\n* Monitor real-time model performance, with support for delayed ground truth/feedback\n* Root cause model failures/performance degradation using tracing and explainability\n* Conduct multi-model performance comparisons\n* Surface drift, data quality, and model fairness/bias metrics&#x20;\n\n","\n{% hint style=""info"" %}\nView this page for more information on surrogate model feature importance values. For an example of logging surrogate model feature importance values values, check out the Arize Surrogate Model Feature Importance tutorial.\n{% endhint %}\n\n! (1).png>)\n\nSurrogate explainability is based on the idea of training a surrogate model to mimic a blackbox model, where a surrogate model is an interpretable model trained to approximate the predictions of a black box model as closely as possible. SHAP values can then be generated from the surrogate model when the blackbox model is not available. The goal is to approximate the predictions of the black box model as closely as possible and generate feature importance values from the interpretable surrogate model.\n\nArize Python SDK (specifically the pandas logger) gives the user the option to pass a flag with their request to send data that would produce SHAP values using the surrogate explainability approach. When the flag is enabled, a tree-based surrogate model is trained using the dataset's features and predictions, and SHAP values are generated from the surrogate model before sending the combined dataset to the Arize platform. The pandas logger can compute surrogate models for regression and binary classification models. For binary classification, the prediction score should have values between 0 and 1.&#x20;\n\nIn this Colab we show how to use the `surrogate_explainability` flag without calculating library to generate feature importance values from a surrogate model using only the prediction outputs from a black box model. Both classification and regression examples are provided and fea

## 7. Compute Ranking Metrics

Now that you know whether each piece of retrieved context is relevant or irrelevant to the corresponding query, you can compute precision@k for k = 1, 2 for each query. This metric tells you what percentage of the retrieved context is relevant to the corresponding query.

precision@k = (# of top-k retrieved documents that are relevant) / (k retrieved documents)

If your precision@2 is greater than zero for a particular query, your LlamaIndex application successfully retrieved at least one relevant piece of context with which to answer the query. If the precision@k is zero for a particular query, that means that no relevant piece of context was retrieved.

Compute precision@k for k = 1, 2 and view the results.

In [ ]:
num_retrieved_documents = 2
num_relevant_documents_array = np.zeros(len(query_df))
for retrieved_context_index in range(0, num_retrieved_documents):
    current_num_retrieved_documents = retrieved_context_index + 1
    num_relevant_documents_array += (
        query_df[f"openai_relevance_{retrieved_context_index}"]
        .map(lambda x: int(x == "relevant"))
        .to_numpy()
    )
    query_df[f"openai_precision_at_{current_num_retrieved_documents}"] = pd.Series(
        num_relevant_documents_array / current_num_retrieved_documents
    )

query_df[
    [
        "openai_relevance_0",
        "openai_relevance_1",
        "openai_precision_at_1",
        "openai_precision_at_2",
    ]
]


## 8. Launch Phoenix

Define your knowledge base dataset with a schema that specifies the meaning of each column (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
database_schema = px.Schema(
    prediction_id_column_name="document_id",
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="text_vector",
        raw_data_column_name="text",
    ),
)
database_ds = px.Dataset(
    dataframe=database_df,
    schema=database_schema,
    name="database",
)

Define your query dataset. Because the query dataframe is in OpenInference format, Phoenix is able to infer the meaning of each column without a user-defined schema by using the `phoenix.Dataset.from_open_inference` class method.

In [ ]:
query_schema = px.Schema(
    prediction_id_column_name=":id.id:",
    timestamp_column_name=":timestamp.iso_8601:",
    prompt_column_names=px.RetrievalEmbeddingColumnNames(
        vector_column_name="prompt_embedding",
        raw_data_column_name=":feature.text:prompt",
        context_retrieval_ids_column_name=":feature.[str].retrieved_document_ids:prompt",
        context_retrieval_scores_column_name=":feature.[float].retrieved_document_scores:prompt",
    ),
    response_column_names=":prediction.text:response",
    tag_column_names=[
        "openai_relevance_0",
        "openai_relevance_1",
        "openai_precision_at_1",
        "openai_precision_at_2",
        "user_feedback",
    ],
)
query_ds = px.Dataset(
    query_df.rename(columns={":feature.[float].embedding:prompt": "prompt_embedding"}),
    schema=query_schema,
    name="query",
)

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(primary=query_ds, corpus=database_ds)


## 9. Surface Problematic Clusters and Data Points

Select `centered_text_vector` in the embeddings section of the Phoenix homepage.

![select center_text_vector in embeddings section](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/select_centered_text_vector.png)

Phoenix helps you:

- reduces the dimensionality of your embeddings for visualization
- color the resulting point cloud using evaluation metrics and other color strategies
- cluster the points and surface up problematic clusters based on whatever metric you care about

Notice that, by default, the data points are colored based on the dataset they belong to (query vs. database) and the clusters are sorted based on "purity," with clusters containing all or mostly query data appearing near the top. If a cluster contains all query data, that's potentially problematic, because it's more likely that your database does not contain relevant context to answer the query.

![notice default coloring and cluster sorting strategies](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/notice_default_color_and_cluster_sorting.png)

Select the top cluster and examine the queries and responses from the chatbot. Notice this cluster contains questions around pricing and cost of the Arize platform. It turns out, the Arize documentation does not contain any information on pricing, so this cluster of queries is far from any database entries.

![select and examine top cluster by purity](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/select_and_examine_top_cluster_by_purity.png)

Now let's investigate clusters of queries/ responses that received low user feedback. Change the coloring of the data by selecting `Color By` > `dimension` > `user_feedback`.

![color by user feedback](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/color_by_user_feedback.png)

Sort your clusters by average user feedback.

![sort clusters by average user feedback](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/sort_clusters_by_average_user_feedback.png)

Sort your clusters in ascending order.

![sort clusters in ascending order](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/sort_clusters_in_ascending_order.png)

Select the top cluster by average user feedback. Notice that the same pricing cluster is being surfaced as problematic, this time, because your users are giving thumbs down to the responses from this cluster.

![select top cluster by average user feedback](http://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/search-and-retrieval/llama-index/select_top_cluster_by_average_user_feedback.png)

Now color your data by "openai_precision@2" and sort your clusters in ascending order by average "openai_precision@2". You should see that each entry of the pricing cluster you investigated before has a precision@2 of 0, meaning that no relevant context was retrieved for these queries. You were able to discover this completely automatically using Phoenix with LLM-assisted ranking metrics.

Congrats! You've identified a problematic cluster of queries. You now have tools at your disposal to investigate clusters of queries where your search and retrieval application is performing poorly based

- query purity
- user feedback
- LLM-assisted ranking metrics

As an actionable next step, you should augment your knowledge base to include information about the pricing and cost of the Arize platform, since your users seem especially interested in this topic.